In [1]:
!pip install -q --no-index --find-links /kaggle/input/ultralytics ultralytics

In [2]:
import os
import pydicom
from PIL import Image
import numpy as np
from multiprocessing import Pool, cpu_count

import sklearn.metrics
import torch
import cv2
import numpy as np 
import pandas as pd 
from tqdm.auto import tqdm

In [3]:
EVAL = False # Change to True to compute the validation score
IMG_DIR = '/images'
FOLD = 0
SAMPLE = False # True for quick debugging
SEVERITIES = ['Normal/Mild', 'Moderate', 'Severe']
LEVELS = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']

SCS_WEIGHTS = ['/kaggle/input/scs-yolov8m/best.pt']

SS_WEIGHTS = ['/kaggle/input/ss-yolov8m/best.pt',
             '/kaggle/input/lsdc-yolo-ssv3/best.pt']

NFN_WEIGHTS = ['/kaggle/input/nfn-yolov8m/best.pt',
              '/kaggle/input/lsdc-yolo-nfnv3/best.pt']

In [4]:
if EVAL:
    import sys
    sys.path.append('/kaggle/input/lsdc-utils')
    from metrics import score as lsdc_scoring

In [5]:
train_val_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')

In [6]:
if EVAL:
    train_xy = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')
    des = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')
else:    
    des = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv')

In [7]:
def read_dcm(src_path):
    dicom_data = pydicom.dcmread(src_path)
    image = dicom_data.pixel_array
    image = (image - image.min()) / (image.max() - image.min() +1e-6) * 255
    return image

def convert_dcm_to_jpg(file_path):
    try:
        # Read the DICOM file
        image_array = read_dcm(file_path)
        
        # Define the output path
        relative_path = os.path.relpath(file_path, start=input_directory)
        output_path = os.path.join(output_directory, relative_path)
        output_path = output_path.replace('.dcm', '.jpg')
                
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Save the image as a JPEG file
        cv2.imwrite(output_path, image_array)
        
        return output_path
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def process_files(dcm_files):
    with Pool(cpu_count()) as pool:
        # Wrap pool.map with tqdm to show the progress bar
        list(tqdm(pool.imap(convert_dcm_to_jpg, dcm_files), total=len(dcm_files)))

def get_dcm_files(directory):
    dcm_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.dcm'):
                dcm_files.append(os.path.join(root, file))
    return dcm_files    

In [8]:
# Replace these with your input and output directories
if not EVAL:
    input_directory = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images'

    output_directory = IMG_DIR

    # Get all .dcm files in the input directory
    dcm_files = get_dcm_files(input_directory)

    # Process the files using multiprocessing
    process_files(dcm_files)

    print(f"Conversion completed. Images saved to {output_directory}")
else:
    if not os.path.exists(IMG_DIR):
        print('Unziping data..')
        !unzip -q -d / /kaggle/input/lsdc-get-all-images/images.zip
        print('Done unziping data')

  0%|          | 0/97 [00:00<?, ?it/s]

Conversion completed. Images saved to /images


In [9]:
if EVAL:
    fold_df = pd.read_csv('/kaggle/input/lsdc-fold-split/5folds.csv')
    test_df = fold_df[fold_df.fold == FOLD]
    
else:
    test_df = os.listdir('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images')
    test_df = pd.DataFrame(test_df, columns=['study_id'])
    test_df['study_id'] = test_df['study_id'].astype(int)
    
test_df = test_df.merge(des, on=['study_id'])

In [10]:
def gen_label_map(CONDITIONS):
    label2id = {}
    id2label = {}
    i = 0
    for cond in CONDITIONS:
        for level in LEVELS:
            for severity in SEVERITIES:
                cls_ = f"{cond.lower().replace(' ', '_')}_{level}_{severity.lower()}"
                label2id[cls_] = i
                id2label[i] = cls_
                i+=1
    return label2id, id2label
                
scs_label2id, scs_id2label = gen_label_map(['Spinal Canal Stenosis'])
ss_label2id, ss_id2label = gen_label_map(['Left Subarticular Stenosis', 'Right Subarticular Stenosis'])
nfn_label2id, nfn_id2label = gen_label_map(['Left Neural Foraminal Narrowing', 'Right Neural Foraminal Narrowing'])

In [11]:
from ultralytics import YOLO

# Load YOLO Model
scs_models = []
for weight in SCS_WEIGHTS:
    scs_models.append(YOLO(weight))
    
ss_models = []
for weight in SS_WEIGHTS:
    ss_models.append(YOLO(weight))
    
nfn_models = []
for weight in NFN_WEIGHTS:
    nfn_models.append(YOLO(weight))

In [12]:
all_label_set = train_val_df.iloc[0, 1:].index.tolist()
scs_label_set = all_label_set[:5]
nfn_label_set = all_label_set[5:15]
ss_label_set = all_label_set[15:]

In [13]:
settings = [
    ('Sagittal T2/STIR', scs_models, scs_id2label, scs_label_set, 0.006),  # 设置为0.3
    ('Axial T2', ss_models, ss_id2label, ss_label_set, 0.007),  # 设置为0.4
    ('Sagittal T1', nfn_models, nfn_id2label, nfn_label_set, 0.008)  # 设置为0.5
]

In [14]:
from collections import defaultdict

In [15]:
pred_rows = []

for modality, models, id2label, label_set, thresh in settings:
    mod_df = test_df[test_df.series_description == modality]
    
    if SAMPLE:
        mod_df = mod_df.sample(20, random_state=610)
    
    # for each study, at each level and condition, get the maximum probability score
    for study_id, group in tqdm(mod_df.groupby('study_id')):
        predictions = defaultdict(list)
        for i, row in group.iterrows():
            # predict on all images from all the series
            series_dir = os.path.join(IMG_DIR, str(row['study_id']), str(row['series_id']))
            for model in models:
                results = model(series_dir, conf=thresh, verbose=False)
                for res in results:
                    for pred_class, conf in zip(res.boxes.cls, res.boxes.conf):
                        pred_class = pred_class.item()
                        conf = conf.item()
                        _class = id2label[pred_class]
                        predictions[_class].append(conf)
        
        # aggregate the result on images to obtain study-level prediction
        for condition in label_set:
            res_dict = {'row_id': f'{study_id}_{condition}' }

            score_vec = []
            for severity in SEVERITIES:
                severity = severity.lower()
                key = f'{condition}_{severity}'
                if len(predictions[key]) > 0:
                    score = np.max(predictions[key])
                else:
                    score = thresh
                score_vec.append(score)
                
            # normalize score to sum to 1
            score_vec = torch.tensor(score_vec)
            score_vec = score_vec / score_vec.sum()

            for idx, severity in enumerate(SEVERITIES):
                res_dict[severity.replace('/', '_').lower()] = score_vec[idx].item()

            pred_rows.append(res_dict)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
pred_df = pd.DataFrame(pred_rows)
pred_df

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.367816,0.616965,0.015219
1,44036939_spinal_canal_stenosis_l2_l3,0.007631,0.415298,0.577071
2,44036939_spinal_canal_stenosis_l3_l4,0.361608,0.441748,0.196644
3,44036939_spinal_canal_stenosis_l4_l5,0.480037,0.248094,0.271869
4,44036939_spinal_canal_stenosis_l5_s1,0.441513,0.487234,0.071253
5,44036939_left_subarticular_stenosis_l1_l2,0.485329,0.501122,0.013549
6,44036939_left_subarticular_stenosis_l2_l3,0.224721,0.261396,0.513883
7,44036939_left_subarticular_stenosis_l3_l4,0.262064,0.242195,0.495741
8,44036939_left_subarticular_stenosis_l4_l5,0.264333,0.467204,0.268463
9,44036939_left_subarticular_stenosis_l5_s1,0.361778,0.418326,0.219896


In [17]:
pred_df.to_csv('submission1.csv', index=False)

# Model 2

In [18]:
%pip install --quiet /kaggle/input/timm_3d_deps/other/initial/9/pydicom/pydicom/pydicom-2.4.4-py3-none-any.whl
%pip install timm_3d --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/9/timm_3d/
%pip install torchio --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/9/torchio/
%pip install itk --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/9/itk/itk
%pip install skorch --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/9/skorch/skorch
%pip install spacecutter --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/9/spacecutter/
%pip install open3d --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/9/open3d

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
data_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/"

In [20]:
import pandas as pd

def retrieve_test_data(data_path):
    test_df = pd.read_csv(data_path + 'test_series_descriptions.csv')

    return test_df

retrieve_test_data(data_path)

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [21]:
import os

def retrieve_image_paths(base_path, study_id, series_id):
    series_dir = os.path.join(base_path, str(study_id), str(series_id))
    images = os.listdir(series_dir)
    image_paths = [os.path.join(series_dir, img) for img in images]
    return image_paths

In [22]:
import open3d as o3d
from pydicom import dcmread
import math
import numpy as np
import cv2
import copy

def read_study_as_pcd(dir_path, series_types_dict=None, downsampling_factor=1, img_size=(256, 256)):
    pcd_overall = o3d.geometry.PointCloud()

    for path in glob.glob(os.path.join(dir_path, "**/*.dcm"), recursive=True):
        dicom_slice = dcmread(path)

        series_id = os.path.basename(os.path.dirname(path))
        study_id = os.path.basename(os.path.dirname(os.path.dirname(path)))
        if series_types_dict is None or int(series_id) not in series_types_dict:
            series_desc = dicom_slice.SeriesDescription
        else:
            series_desc = series_types_dict[int(series_id)]
            series_desc = series_desc.split(" ")[-1]

        x_orig, y_orig = dicom_slice.pixel_array.shape
        img = np.expand_dims(cv2.resize(dicom_slice.pixel_array, img_size, interpolation=cv2.INTER_AREA), -1)
        x, y, z = np.where(img)

        downsampling_factor_iter = max(downsampling_factor, int(math.ceil(len(x) / 6e6)))

        index_voxel = np.vstack((x, y, z))[:, ::downsampling_factor_iter]
        grid_index_array = index_voxel.T
        pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(grid_index_array.astype(np.float64)))

        vals = np.expand_dims(img[x, y, z][::downsampling_factor_iter], -1)
        if series_desc == "T1":
            vals = np.pad(vals, ((0, 0), (0, 2)))
        elif series_desc == "T2":
            vals = np.pad(vals, ((0, 0), (1, 1)))
        elif series_desc == "T2/STIR":
            vals = np.pad(vals, ((0, 0), (2, 0)))
        else:
            raise ValueError(f"Unknown series desc: {series_desc}")

        pcd.colors = o3d.utility.Vector3dVector(vals.astype(np.float64))

        dX, dY = dicom_slice.PixelSpacing
        dZ = dicom_slice.SliceThickness

        X = np.array(list(dicom_slice.ImageOrientationPatient[:3]) + [0]) * dX
        Y = np.array(list(dicom_slice.ImageOrientationPatient[3:]) + [0]) * dY

        for z in range(int(dZ)):
            pos = list(dicom_slice.ImagePositionPatient)
            if series_desc == "T2":
                pos[-1] += z
            else:
                pos[0] += z
            S = np.array(pos + [1])

            transform_matrix = np.array([X, Y, np.zeros(len(X)), S]).T
            transform_matrix = transform_matrix @ np.matrix(
                [[0, y_orig / img_size[1], 0, 0],
                 [x_orig / img_size[0], 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]]
            )

            pcd_overall += copy.deepcopy(pcd).transform(transform_matrix)

    return pcd_overall



def read_study_as_voxel_grid(dir_path, series_type_dict=None, downsampling_factor=1, img_size=(256, 256)):
    pcd_overall = read_study_as_pcd(dir_path,
                                    series_types_dict=series_type_dict,
                                    downsampling_factor=downsampling_factor,
                                    img_size=img_size)
    box = pcd_overall.get_axis_aligned_bounding_box()

    max_b = np.array(box.get_max_bound())
    min_b = np.array(box.get_min_bound())

    pts = (np.array(pcd_overall.points) - (min_b)) * (
                (img_size[0] - 1, img_size[0] - 1, img_size[0] - 1) / (max_b - min_b))
    coords = np.round(pts).astype(np.int32)
    vals = np.array(pcd_overall.colors, dtype=np.float16)

    grid = np.zeros((3, img_size[0], img_size[0], img_size[0]), dtype=np.float16)
    indices = coords[:, 0], coords[:, 1], coords[:, 2]

    np.maximum.at(grid[0], indices, vals[:, 0])
    np.maximum.at(grid[1], indices, vals[:, 1])
    np.maximum.at(grid[2], indices, vals[:, 2])


    return grid

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [23]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchio as tio
import torch.nn as nn
import pydicom

CONDITIONS = {
    "Sagittal T2/STIR": ["Spinal Canal Stenosis"],
    "Axial T2": ["Left Subarticular Stenosis", "Right Subarticular Stenosis"],
    "Sagittal T1": ["Left Neural Foraminal Narrowing", "Right Neural Foraminal Narrowing"],
}


class PatientLevelTestset(Dataset):
    def __init__(self,
                 base_path: str,
                 dataframe: pd.DataFrame,
                 transform_3d=None):
        self.base_path = base_path

        self.dataframe = (dataframe[['study_id', "series_id", "series_description"]]
                          .drop_duplicates())

        self.subjects = self.dataframe[['study_id']].drop_duplicates().reset_index(drop=True)
        self.series_descs = {e[0]: e[1] for e in self.dataframe[["series_id", "series_description"]].drop_duplicates().values}

        self.transform_3d = transform_3d

    def __len__(self):
        return len(self.subjects)

    def __getitem__(self, index):
        curr = self.subjects.iloc[index]
        study_path = os.path.join(self.base_path, str(curr["study_id"]))

        study_images = read_study_as_voxel_grid(study_path, self.series_descs)

        if self.transform_3d is not None:
            study_images = self.transform_3d(torch.FloatTensor(study_images))  # .data
            return study_images.to(torch.half), str(curr["study_id"])

        return torch.HalfTensor(study_images), str(curr["study_id"])


In [24]:
transform_3d = tio.Compose([
    tio.RescaleIntensity([0, 1]),
])

In [25]:
def create_subject_level_testset_and_loader(df: pd.DataFrame,
                                             transform_3d,
                                             base_path: str,
                                             batch_size=1,
                                             num_workers=0):
    testset = PatientLevelTestset(base_path, df, transform_3d=transform_3d)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return testset, test_loader

In [26]:
import os

data = retrieve_test_data(data_path)
dataset, dataloader = create_subject_level_testset_and_loader(data, transform_3d, os.path.join(data_path, "test_images"))

In [27]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [28]:
import torch.nn as nn
import timm_3d
from spacecutter import *
from spacecutter.losses import *
from spacecutter.models import *
from spacecutter.callbacks import *


class CNN_Model_3D_Multihead(nn.Module):
    def __init__(self,
                 backbone="efficientnet_lite0",
                 in_chans=1,
                 out_classes=5,
                 cutpoint_margin=0.15,
                 pretrained=False):
        super(CNN_Model_3D_Multihead, self).__init__()
        self.out_classes = out_classes

        self.encoder = timm_3d.create_model(
            backbone,
            features_only=False,
            drop_rate=0,
            drop_path_rate=0,
            pretrained=pretrained,
            in_chans=in_chans,
            global_pool="max"
        )
        if "efficientnet" in backbone:
            head_in_dim = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Sequential(
                nn.LayerNorm(head_in_dim),
                nn.Dropout(0),
            )

        elif "vit" in backbone:
            self.encoder.head.drop = nn.Dropout(0)
            head_in_dim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()

        self.heads = nn.ModuleList(
            [nn.Sequential(
                nn.Linear(head_in_dim, 1),
                LogisticCumulativeLink(3)
            ) for i in range(out_classes)]
        )

        self.ascension_callback = AscensionCallback(margin=cutpoint_margin)

    def forward(self, x):
        feat = self.encoder(x)
        return torch.swapaxes(torch.stack([head(feat) for head in self.heads]), 0, 1)

    def _ascension_callback(self):
        for head in self.heads:
            self.ascension_callback.clip(head[-1])

In [29]:
model = CNN_Model_3D_Multihead(backbone="maxvit_rmlp_tiny_rw_256", in_chans=3, out_classes=25).to(device)
model.load_state_dict(torch.load("/kaggle/input/rsna-2024/pytorch/vit_voxel_v2/5/maxvit_rmlp_tiny_rw_256_256_v2_fold_3_32.pt"))

/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<All keys matched successfully>

In [30]:
CONDITIONS = {
    "Sagittal T2/STIR": ["spinal_canal_stenosis"],
    "Axial T2": ["left_subarticular_stenosis", "right_subarticular_stenosis"],
    "Sagittal T1": ["left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing"],
}

ALL_CONDITIONS = sorted(["spinal_canal_stenosis", "left_subarticular_stenosis", "right_subarticular_stenosis", "left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing"])
LEVELS = ["l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"]

results_df = pd.DataFrame({"row_id":[], "normal_mild": [], "moderate": [], "severe": []})

ALL_CONDITIONS

['left_neural_foraminal_narrowing',
 'left_subarticular_stenosis',
 'right_neural_foraminal_narrowing',
 'right_subarticular_stenosis',
 'spinal_canal_stenosis']

In [31]:
# Pre-populate results df
import glob
import os

study_ids = glob.glob("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/*")
study_ids = [os.path.basename(e) for e in study_ids]

results_df = pd.DataFrame({"row_id":[], "normal_mild": [], "moderate": [], "severe": []})
for study_id in study_ids:
    for condition in ALL_CONDITIONS:
        for level in LEVELS:
            row_id = f"{study_id}_{condition}_{level}"
            results_df = results_df._append({"row_id": row_id, "normal_mild": 1/3, "moderate": 1/3, "severe": 1/3}, ignore_index=True)

In [32]:
dataset.dataframe

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [33]:
import torch
from torch.cuda.amp import autocast
import time

start_time = time.time()

with torch.no_grad():
    with autocast(dtype=torch.float16):
        model.eval()

        for images, study_id in dataloader:
            output = model(images.to(device))
            for i, batch_out in enumerate(output):
                batch_out = output.cpu().numpy()[i]
                for index, level in enumerate(batch_out):
                    row_id = f"{study_id[i]}_{ALL_CONDITIONS[index // 5]}_{LEVELS[index % 5]}"
                    results_df.loc[results_df.row_id == row_id,'normal_mild'] = level[0]
                    results_df.loc[results_df.row_id == row_id,'moderate'] = level[1]
                    results_df.loc[results_df.row_id == row_id,'severe'] = level[2]
                
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.642016649246216 seconds ---


In [34]:
results_df

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.416031,0.406802,0.177167
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.551157,0.380298,0.068545
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.303012,0.541103,0.155885
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.146881,0.501662,0.351457
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.050625,0.230831,0.718544
5,44036939_left_subarticular_stenosis_l1_l2,0.593061,0.324840,0.082099
6,44036939_left_subarticular_stenosis_l2_l3,0.238710,0.521463,0.239828
7,44036939_left_subarticular_stenosis_l3_l4,0.209273,0.538242,0.252485
8,44036939_left_subarticular_stenosis_l4_l5,0.150982,0.490380,0.358637
9,44036939_left_subarticular_stenosis_l5_s1,0.050674,0.254995,0.694331


In [35]:
results_df.to_csv('submission2.csv', index=False)

In [36]:
sm50 = pd.read_csv('submission1.csv')
sm51 = pd.read_csv('submission2.csv')
display(sm50,sm51)
sms = pd.merge(sm50,sm51, on=['row_id'])
sms['normal_mild'] = sms['normal_mild_x'] *0.7 + 0.3* sms['normal_mild_y']
sms['moderate']    = sms['moderate_x']    *0.7 + 0.3* sms['moderate_y']
sms['severe']      = sms['severe_x']      *0.7 + 0.3* sms['severe_y']

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.367816,0.616965,0.015219
1,44036939_spinal_canal_stenosis_l2_l3,0.007631,0.415298,0.577071
2,44036939_spinal_canal_stenosis_l3_l4,0.361608,0.441748,0.196644
3,44036939_spinal_canal_stenosis_l4_l5,0.480037,0.248094,0.271869
4,44036939_spinal_canal_stenosis_l5_s1,0.441513,0.487234,0.071253
5,44036939_left_subarticular_stenosis_l1_l2,0.485329,0.501122,0.013549
6,44036939_left_subarticular_stenosis_l2_l3,0.224721,0.261396,0.513883
7,44036939_left_subarticular_stenosis_l3_l4,0.262064,0.242195,0.495741
8,44036939_left_subarticular_stenosis_l4_l5,0.264333,0.467204,0.268463
9,44036939_left_subarticular_stenosis_l5_s1,0.361778,0.418326,0.219896


,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.416031,0.406802,0.177167
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.551157,0.380298,0.068545
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.303012,0.541103,0.155885
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.146881,0.501662,0.351457
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.050625,0.230831,0.718544
5,44036939_left_subarticular_stenosis_l1_l2,0.593061,0.324840,0.082099
6,44036939_left_subarticular_stenosis_l2_l3,0.238710,0.521463,0.239828
7,44036939_left_subarticular_stenosis_l3_l4,0.209273,0.538242,0.252485
8,44036939_left_subarticular_stenosis_l4_l5,0.150982,0.490380,0.358637
9,44036939_left_subarticular_stenosis_l5_s1,0.050674,0.254995,0.694331


In [37]:
sub = sms[['row_id','normal_mild','moderate','severe']]
sub.to_csv('submission.csv', index=False, float_format='%.7f')
display(sub)

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.486584,0.490778,0.022638
1,44036939_spinal_canal_stenosis_l2_l3,0.148683,0.417498,0.433820
2,44036939_spinal_canal_stenosis_l3_l4,0.471977,0.380232,0.147791
3,44036939_spinal_canal_stenosis_l4_l5,0.487544,0.294051,0.218405
4,44036939_spinal_canal_stenosis_l5_s1,0.532435,0.400372,0.067193
5,44036939_left_subarticular_stenosis_l1_l2,0.517649,0.448237,0.034114
6,44036939_left_subarticular_stenosis_l2_l3,0.228917,0.339416,0.431667
7,44036939_left_subarticular_stenosis_l3_l4,0.246227,0.331009,0.422764
8,44036939_left_subarticular_stenosis_l4_l5,0.230328,0.474157,0.295515
9,44036939_left_subarticular_stenosis_l5_s1,0.268447,0.369326,0.362227
